In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

## for display
from IPython.display import display

Final: This problem is about solving "graph coloring problem" using grover's algorithm 

We will use a Total of 28 qubits
Each District's color is Represented with 2 qubits each, where 00: A, 01: B, 10: C, 11: D.
Since we have total of 7 districs that need to be colored, we use total of 14 qubits.

We also need qubits for comparing the colors between the districts that are connected, and we have total of 13 edges so we need 13 qubits.(Actually we have more edges but we can reduce them by superpositioning the inputs in the initialzation step)

Finally we need one qubit for the oracle so that we can flip the correct states.

In [2]:
# Create a Quantum Circuit 
qc = QuantumCircuit(28,14)
theta = 2 * np.arccos(1 / np.sqrt(3))

Initialization step is where we can reduce the number of edges from 19 to 13 edges.
For example, district 0 is initially connected with color A, therefore we know that district 0 cannot have the color A. 
If we can restrict the colors of district 0 such that it can only be colored with colors B,C,D, we can reduce the edge that connects district 0 with color A.
We can restrict the available colors by constructing a superpositioned state that only has 01, 10, 11.
The superpositioned state is made using a controlled Hadamard Gate such that A|01> + B|10> + C|11>.
And A = B = C, so we use a rotational Y gate to match this condition and we get A = 1/sqrt(3) = cos(theta/2) 

We repeat this process for evey district that is initially connected to a color.

In [3]:
# superposiiton available districts for using less qubits
def initialize(qc):
    #district 0: 01, 10, 11
    qc.ry(theta,0)
    qc.ch(0, 1)
    qc.x(1)
    
    #district 1: 00, 10, 11
    qc.ry(theta,2)
    qc.ch(2, 3)
    
    #district 2: 01, 11
    qc.h(4)
    qc.x(5)
    
    #district 3: 01, 10, 11
    qc.ry(theta,6)
    qc.ch(6, 7)
    qc.x(7)
    
    #district 4: 00, 10, 11
    qc.ry(theta,8)
    qc.ch(8, 9)
    
    #district 5: 01, 10, 00
    qc.ry(theta,10)
    qc.ch(10, 11)
    qc.x(10)
    
    #district 6: 01, 10, 00
    qc.ry(theta,12)
    qc.ch(12, 13)
    qc.x(12)


In [4]:
#edge_list = ([0,1],[0,2],[0,3],[1,3],[1,4],[2,3],[2,5],[2,6],[3,4],[3,5],[3,6],[4,6],[5,6)

Now we need gates to check whether the two districts have the same Color or not
The IBM Solution checks whether these two districts have the same color using ccccx gates
For example, if both districts are colored with color D(11), they use an ordinary ccccx gate to find out this pair has the same color, and for color A(00) they add not gate to every control bit and use the ccccx gate and etc.....(4 ccccx gates in total)

However, these 4 ccccx gates can be reduced using a hypercube into 4 ccx gates.

In [5]:
def color_check(qc, a,b,c,d, result):
    qc.barrier()
    qc.ccx(a,b,result)
    
    qc.x(c)
    qc.ccx(b,c,result)
    qc.x(c)
    
    qc.x(c)
    qc.x(d)
    qc.ccx(c,d,result)
    qc.x(c)
    qc.x(d)
    
    qc.x(d)
    qc.ccx(a,d,result)
    qc.x(d)
    
    qc.barrier()



In [6]:
def color_check_inverse(qc, a,b,c,d, result):
    qc.barrier()
    qc.x(d)
    qc.ccx(a,d,result)
    qc.x(d)
    
    qc.x(c)
    qc.x(d)
    qc.ccx(c,d,result)
    qc.x(c)
    qc.x(d)
    
    qc.x(c)
    qc.ccx(b,c,result)
    qc.x(c)
    
    qc.ccx(a,b,result)
    
   
    
    qc.barrier()

This is my version of the color check implementation
Instead of directly checking the colors, we can easily just check whether the two districts have the same values or not.
For example, if district 0 is ab and district 1 is cd and these two are connected, we can check whether a,c are equal, b,d are equal and if both are true, we know that these are the same colors.

In [7]:
def color_check_1(qc, a,b,c,d, result):
    qc.barrier()
    qc.cx(a,c)
    qc.cx(b,d)
    qc.x(c)
    qc.x(d)
    qc.ccx(c,d,result)
    qc.x(d)
    qc.x(c)
    qc.cx(b,d)
    qc.cx(a,c)
    qc.barrier()

In [8]:
def color_check_inverse_1(qc, a,b,c,d, result):
    qc.barrier()
    qc.cx(a,c)
    qc.cx(b,d)
    qc.x(c)
    qc.x(d)
    qc.ccx(c,d,result)
    qc.x(d)
    qc.x(c)
    qc.cx(b,d)
    qc.cx(a,c)
    qc.barrier()

Now we can make the oracle that flips the phase of the correct state
Color check determines whether they have the same colors, and if they have the same colors, the result flipped with a "not" operation
Therefore, we have to make the result qubits into 1 in the first place, such that when its not done a "not" operation, it is indicated as 1, meaning that they don't have the same colors

In [9]:
def oracle(qc):
    for i in range(14, 27):#14~26, 
        qc.x(i)
    color_check_1(qc, 0, 1, 2, 3, 14)
    color_check_1(qc, 0, 1, 4, 5, 15)
    color_check_1(qc, 0, 1, 6, 7, 16)
    color_check_1(qc, 2, 3, 6, 7, 17)
    color_check_1(qc, 2, 3, 8, 9, 18)
    color_check_1(qc, 4, 5, 6, 7, 19)
    color_check_1(qc, 4, 5, 10, 11, 20)
    color_check_1(qc, 4, 5, 12, 13, 21)
    color_check_1(qc, 6, 7, 8, 9, 22)
    color_check_1(qc, 6, 7, 10, 11, 23)
    color_check_1(qc, 6, 7, 12, 13, 24)
    color_check_1(qc, 8, 9, 12, 13, 25)
    color_check_1(qc, 10, 11, 12, 13, 26)

    qc.barrier()
    qc.x(27)
    qc.h(27)
    qc.mcx([14, 15, 16, 17, 18, 19, 20, 21, 22, 23,24, 25, 26], 27)
    qc.h(27)
    qc.x(27)
    
    qc.barrier()
    
    color_check_inverse_1(qc, 10, 11, 12, 13, 26)
    color_check_inverse_1(qc, 8, 9, 12, 13, 25)
    color_check_inverse_1(qc, 6, 7, 12, 13, 24)
    color_check_inverse_1(qc, 6, 7, 10, 11, 23)
    color_check_inverse_1(qc, 6, 7, 8, 9, 22)
    color_check_inverse_1(qc, 4, 5, 12, 13, 21)
    color_check_inverse_1(qc, 4, 5, 10, 11, 20)
    color_check_inverse_1(qc, 4, 5, 6, 7, 19)
    color_check_inverse_1(qc, 2, 3, 8, 9, 18)
    color_check_inverse_1(qc, 2, 3, 6, 7, 17)
    color_check_inverse_1(qc, 0, 1, 6, 7, 16)
    color_check_inverse_1(qc, 0, 1, 4, 5, 15)
    color_check_inverse_1(qc, 0, 1, 2, 3, 14)
    
    
    for i in range(14, 27):#14~26
        qc.x(i)

In [10]:
def initialize_inverse(qc):
    #district 0: 01, 10, 11
    qc.x(1)
    qc.ch(0, 1)
    qc.ry(-theta,0)
    
    #district 1: 00, 10, 11
    qc.ch(2, 3)
    qc.ry(-theta,2)
    
    #district 2: 01, 11
    qc.x(5)
    qc.h(4)
    
    #district 3: 01, 10, 11
    qc.x(7)
    qc.ch(6, 7)
    qc.ry(-theta,6)
    
    #district 4: 00, 10, 11
    qc.ch(8, 9)
    qc.ry(-theta,8)
    
    #district 5: 01, 10, 00
    qc.x(10)
    qc.ch(10, 11)
    qc.ry(-theta,10)
    
    #district 6: 01, 10, 00
    qc.x(12)
    qc.ch(12, 13)
    qc.ry(-theta,12)

This part is the Grover's Diffusion process

In [11]:
def diffusion(qc):
    initialize_inverse(qc)
    for i in range(0, 14):#0~13
        qc.x(i)
    qc.h(13)
    qc.mcx([0,1,2,3,4,5,6,7,8,9,10,11,12],13)
    qc.h(13)
    for i in range(0, 14):#0~13
        qc.x(i)
    initialize(qc)

Now, since we have all the pieces, we can assemble them to make a Grover's Algorithm circuit with iteration 5(iteration value is given be the quantum challenge document but it can be calculated easily)

In [12]:
initialize(qc)

for i in range(5):
    oracle(qc)
    diffusion(qc)
    
qc.barrier()
qc.measure([0,1,2,3,4,5,6,7,8,9,10,11,12,13], [0,1,2,3,4,5,6,7,8,9,10,11,12,13])
#display(qc.draw())

In [13]:
%%time
compiled_circuit = transpile(qc, simulator)

# Execute the circuit on the qasm simulator
job = simulator.run(compiled_circuit, shots=8000)

# Grab results from the job
result = job.result()
#print(result)
# Returns counts
counts = result.get_counts(compiled_circuit)


CPU times: user 1h 3min 20s, sys: 12.4 s, total: 1h 3min 33s
Wall time: 20min 39s


We get the results of the to 15 counts and we get the 9 solutions as below!

In [14]:
score_sorted = sorted(counts.items(), key=lambda x:x[1], reverse=True)
final_score = score_sorted[0:15]
final_score

[('00101101110010', 540),
 ('00011110110001', 530),
 ('01001110110001', 529),
 ('10000001111110', 524),
 ('10001101110010', 521),
 ('00010110111101', 487),
 ('00010111100001', 480),
 ('01000010111101', 479),
 ('00010110110001', 468),
 ('00010101100010', 8),
 ('00010011100001', 8),
 ('01000011100011', 8),
 ('01010101100111', 8),
 ('00000101111101', 7),
 ('00100111111101', 7)]

In [ ]:
#original answer
[('00010110110001', 577),
 ('01001110110001', 521),
 ('01000010111101', 518),
 ('10001101110010', 510),
 ('00101101110010', 508),
 ('00011110110001', 503),
 ('00010111100001', 500),
 ('10000001111110', 500),
 ('00010110111101', 455),
 ('01001101100101', 9),
 ('10010111111101', 9),
 ('00100110101101', 8),
 ('10100111101111', 8),
 ('00100101111111', 8),
 ('10000101100111', 8)]

In [ ]:
#with my answer
[('00101101110010', 540),
 ('00011110110001', 530),
 ('01001110110001', 529),
 ('10000001111110', 524),
 ('10001101110010', 521),
 ('00010110111101', 487),
 ('00010111100001', 480),
 ('01000010111101', 479),
 ('00010110110001', 468),
 ('00010101100010', 8),
 ('00010011100001', 8),
 ('01000011100011', 8),
 ('01010101100111', 8),
 ('00000101111101', 7),
 ('00100111111101', 7)]

I tried calculating the circuit's cost but the kernel died every time when I tried to unroll this circuit